In [9]:
import tensorflow
from tensorflow import keras
import pandas as pd
import numpy as np

In [2]:
base= keras.models.load_model('../model/exports/itcStockPredLSTM(v2)/')

In [5]:
df= pd.read_csv('../dataset/ITC.NS-test.csv')

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler= MinMaxScaler(feature_range=(0,1))

In [7]:
trainingSetScaled= scaler.fit_transform(df['Close'].values.reshape(-1,1))

In [10]:
xTRAIN = []
yTRAIN = []

for i in range(60, len(trainingSetScaled)):
    xTRAIN.append(trainingSetScaled[i-60:i, 0])
    yTRAIN.append(trainingSetScaled[i, 0])
    
xTRAIN, yTRAIN= np.array(xTRAIN), np.array(yTRAIN)

In [11]:
xTRAIN= np.reshape(xTRAIN, newshape=(xTRAIN.shape[0], xTRAIN.shape[1], 1))

In [27]:
xTRAIN.shape

(434, 60, 1)

In [30]:
regressor= keras.models.Sequential([
    keras.layers.LSTM(units=64, return_sequences= True, input_shape= (xTRAIN.shape[1], 1)),
    keras.layers.Dropout(rate= 0.2),
    base,
    keras.layers.LSTM(units=50, return_sequences=True), 
    keras.layers.Dropout(rate= 0.2),
    keras.layers.Dense(units= 1)
])

ValueError: Exception encountered when calling layer "sequential" (type Sequential).

Input 0 of layer "lstm" is incompatible with the layer: expected shape=(None, None, 1), found shape=(None, 60, 64)

Call arguments received by layer "sequential" (type Sequential):
  • inputs=tf.Tensor(shape=(None, 60, 64), dtype=float32)
  • training=False
  • mask=None

In [21]:
model.layers[4].trainable= False

In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 60, 64)            16896     
                                                                 
 dropout_6 (Dropout)         (None, 60, 64)            0         
                                                                 
 lstm_7 (LSTM)               (None, 50)                23000     
                                                                 
 dropout_7 (Dropout)         (None, 50)                0         
                                                                 
 sequential (Sequential)     (None, 1)                 60147     
                                                                 
 dense_1 (Dense)             (None, 1)                 2         
                                                                 
Total params: 100,045
Trainable params: 39,898
Non-tra

In [23]:
regressor.compile(optimizer= 'adam', loss= keras.losses.mean_squared_error)

In [24]:
regressor.fit(x= xTRAIN, y=yTRAIN, batch_size= 32, epochs= 7)

Epoch 1/7


ValueError: in user code:

    File "C:\Users\Rohan\miniconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Rohan\miniconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Rohan\miniconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Rohan\miniconda3\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Rohan\miniconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Rohan\miniconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Exc